# 第四章 变形

###### 前期准备

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('joyful-pandas-master/data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、透视表
### 1、pivot
一般状态下，数据在DataFrame会以压缩（stacked）状态存放，例如上面的Gender，两个类别被叠在一列中，pivot函数可将某一列作为新的cols：

In [3]:
df.pivot(index='ID',columns='Gender',values='Height').head()  # 设行列名，变成一个新的DataFrame

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


然而pivot函数具有很强的局限性，除了功能上较少之外，还不允许values中出现重复的行列索引对（pair），例如下面的语句就会报错：

In [6]:
# df.pivot(index='School',columns='Gender',values='Height').head()
# ValueError: Index contains duplicate entries, cannot reshape

因此，更多的时候会选择使用强大的pivot_table函数

### 2. pivot_table

In [8]:
pd.pivot_table(df,index='ID',columns='Gender',values='Height').head()

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


但是在速度上，由于功能更多，自然是比不上原来的pivot函数的。

In [9]:
%timeit df.pivot(index='ID',columns='Gender',values='Height')
%timeit pd.pivot_table(df,index='ID',columns='Gender',values='Height')

1.82 ms ± 58.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.45 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Pandas中提供了各种选项，下面介绍常用参数：
#### ① aggfunc：对组内进行聚合统计，可传入各类函数，默认为'mean'

In [10]:
pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc=['mean','sum']).head()

mean               sum      
Gender           F           M     F     M
School                                    
S_1     173.125000  178.714286  1385  1251
S_2     173.727273  172.000000  1911  1548

#### ② margins：汇总边际状态

In [11]:
pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc=['mean','sum'],margins=True).head()
#margins_name可以设置名字，默认为'All'

mean                           sum            
Gender           F           M         All     F     M   All
School                                                      
S_1     173.125000  178.714286  175.733333  1385  1251  2636
S_2     173.727273  172.000000  172.950000  1911  1548  3459
All     173.473684  174.937500  174.142857  3296  2799  6095

#### ③ 行、列、值都可以为多级

In [12]:
pd.pivot_table(df,index=['School','Class'], columns=['Gender','Address'], values=['Height','Weight'])

Height                                                        \
Gender              F                                                     M   
Address      street_1 street_2 street_4 street_5 street_6 street_7 street_1   
School Class                                                                  
S_1    C_1        NaN    179.5    159.0      NaN      NaN      NaN    173.0   
       C_2        NaN      NaN    176.0    162.0    167.0      NaN      NaN   
       C_3      175.0      NaN      NaN    187.0      NaN      NaN      NaN   
S_2    C_1        NaN      NaN      NaN    159.0    161.0      NaN      NaN   
       C_2        NaN      NaN      NaN      NaN      NaN    188.5    175.0   
       C_3        NaN      NaN    157.0      NaN    164.0    190.0      NaN   
       C_4        NaN    176.0      NaN      NaN    175.5      NaN      NaN   

                                         ...   Weight                    \
Gender                                   ...        F                     
Address      street_2 street_4 street_5  ... street_4 street_5 street_6   
School Class                             ...                              
S_1    C_1      186.0      NaN      NaN  ...     64.0      NaN      NaN   
       C_2        NaN      NaN    188.0  ...     94.0     63.0     63.0   
       C_3      195.0    161.0      NaN  ...      NaN     69.0      NaN   
S_2    C_1        NaN    163.5      NaN  ...      NaN     97.0     61.0   
       C_2        NaN    155.0    193.0  ...      NaN      NaN      NaN   
       C_3        NaN    187.0    171.0  ...     78.0      NaN     81.0   
       C_4        NaN      NaN      NaN  ...      NaN      NaN     57.0   

                                                                             
Gender                       M                                               
Address      street_7 street_1 street_2 street_4 street_5 street_6 street_7  
School Class                                                                 
S_1    C_1        NaN     63.0     82.0      NaN      NaN      NaN      NaN  
       C_2        NaN      NaN      NaN      NaN     68.0     53.0      NaN  
       C_3        NaN      NaN     70.0     68.0      NaN      NaN     82.0  
S_2    C_1        NaN      NaN      NaN     71.0      NaN      NaN     84.0  
       C_2       76.5     74.0      NaN     91.0    100.0      NaN      NaN  
       C_3       99.0      NaN      NaN     73.0     88.0      NaN      NaN  
       C_4        NaN      NaN      NaN      NaN      NaN      NaN     82.0  

[7 rows x 24 columns]

### 3. crosstab（交叉表）

交叉表是一种特殊的透视表，典型的用途如分组统计，如现在想要统计关于街道和性别分组的**频数**：

In [13]:
pd.crosstab(index=df['Address'],columns=df['Gender'])

Gender,F,M
Address,,
street_1,1,2
street_2,4,2
street_4,3,5
street_5,3,3
street_6,5,1
street_7,3,3


#### 交叉表的功能也很强大（但目前还不支持多级分组），下面说明一些重要参数：
#### ① values和aggfunc：分组对某些数据进行聚合操作，这两个参数必须成对出现

In [18]:
pd.crosstab(index=df['Address'],columns=df['Gender'], values=np.random.randint(1,20,df.shape[0]), aggfunc='min')

Gender,F,M
Address,,
street_1,4,7
street_2,3,1
street_4,9,4
street_5,2,6
street_6,1,8
street_7,2,1


默认参数如下：

In [21]:
pd.crosstab(index=df['Address'],columns=df['Gender'],values=1,aggfunc='count')

Gender,F,M
Address,,
street_1,1,2
street_2,4,2
street_4,3,5
street_5,3,3
street_6,5,1
street_7,3,3


#### ② 除了边际参数margins外，还引入了normalize参数（求百分比），可选'all','index','columns'参数值，也就是对全体、行或列求百分比。

In [25]:
pd.crosstab(index=df['Address'],columns=df['Gender'],normalize='all',margins=True)

Gender,F,M,All
Address,,,
street_1,0.028571,0.057143,0.085714
street_2,0.114286,0.057143,0.171429
street_4,0.085714,0.142857,0.228571
street_5,0.085714,0.085714,0.171429
street_6,0.142857,0.028571,0.171429
street_7,0.085714,0.085714,0.171429
All,0.542857,0.457143,1.000000


## 二、其他变形方法
### 1、melt

melt函数可以认为是pivot函数的逆操作，将unstacked状态的数据，压缩成stacked，使“宽”的DataFrame变“窄”

In [26]:
df_m = df[['ID','Gender','Math']]
df_m.head()

,ID,Gender,Math
0,1101,M,34.0
1,1102,F,32.5
2,1103,M,87.2
3,1104,F,80.4
4,1105,F,84.8


In [28]:
pivoted = df.pivot(index='ID',columns='Gender',values='Math')
pivoted.head()

Gender,F,M
ID,,
1101,NaN,34.0
1102,32.5,NaN
1103,NaN,87.2
1104,80.4,NaN
1105,84.8,NaN


melt函数中的id_vars表示需要保留的列，value_vars表示需要stack的一组列，value_name是value_vars对应的值的列名。

详细可以看：https://pandas.pydata.org/docs/reference/api/pandas.melt.html

In [30]:
result = pivoted.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math').dropna().set_index('ID').sort_index()
result.head()

,Gender,Math
ID,,
1101,M,34.0
1102,F,32.5
1103,M,87.2
1104,F,80.4
1105,F,84.8


In [31]:
result.equals(df_m.set_index('ID'))

True

### 2、压缩与展开
#### （1）stack：这是最基础的变形函数，总共只有两个参数：level和dropna

In [35]:
df_s = pd.pivot_table(df,index=['Class','ID'],columns='Gender',values=['Height','Weight'])
df_s.groupby('Class').head(2)

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
C_2   1201    NaN  188.0    NaN  68.0
      1202  176.0    NaN   94.0   NaN
C_3   1301    NaN  161.0    NaN  68.0
      1302  175.0    NaN   57.0   NaN
C_4   2401  192.0    NaN   62.0   NaN
      2402    NaN  166.0    NaN  82.0

In [36]:
df_stacked = df_s.stack()  # 默认将列往行压缩，从后往前。
df_stacked.groupby('Class').head(2)

Height  Weight
Class ID   Gender                
C_1   1101 M        173.0    63.0
      1102 F        192.0    73.0
C_2   1201 M        188.0    68.0
      1202 F        176.0    94.0
C_3   1301 M        161.0    68.0
      1302 F        175.0    57.0
C_4   2401 F        192.0    62.0
      2402 M        166.0    82.0

当将所有列压入行之后，就变成Series了，比如下一个例子：

In [37]:
ddd = df_stacked.stack()
ddd.groupby('Class').head(2)

Class  ID    Gender        
C_1    1101  M       Height    173.0
                     Weight     63.0
C_2    1201  M       Height    188.0
                     Weight     68.0
C_3    1301  M       Height    161.0
                     Weight     68.0
C_4    2401  F       Height    192.0
                     Weight     62.0
dtype: float64

**结论：**stack函数可以看做将横向的索引放到纵向，因此功能类似与melt，参数level可指定变化的列索引是哪一层（或哪几层，需要列表）

In [39]:
df_stacked = df_s.stack(level=0)  # 这里将第一层横向索引放到纵向。这个参数默认为level，所以也可以省略。
# df_stacked = df_s.stack(0)
df_stacked.groupby('Class').head(2)

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
C_2   1201 Height    NaN  188.0
           Weight    NaN   68.0
C_3   1301 Height    NaN  161.0
           Weight    NaN   68.0
C_4   2401 Height  192.0    NaN
           Weight   62.0    NaN

这里说一下dropna参数，默认是True。这个参数是用来删除缺失值的，这个例子不是很好，展示不出删除缺失值，但是可以看下面分享的链接，有一个例子比较明显的展示了dropna是怎么删除缺失值的。

如果dropna=True，那么就是会将缺失值删除，若dropna=False，则会保留缺失值。

In [44]:
qwe = df_s.stack(level=0, dropna = True) 
qwe.groupby('Class').head(2)

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
C_2   1201 Height    NaN  188.0
           Weight    NaN   68.0
C_3   1301 Height    NaN  161.0
           Weight    NaN   68.0
C_4   2401 Height  192.0    NaN
           Weight   62.0    NaN

参考学习：https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html?highlight=stack#pandas.DataFrame.stack

#### (2) unstack：stack的逆函数，功能上类似于pivot_table。
表达式：DataFrame.unstack(self，level = -1，fill_value = None)

In [52]:
df_stacked.head()

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
      1102 Height  192.0    NaN
           Weight   73.0    NaN
      1103 Height    NaN  186.0

**结论：**这个unstack就是相当于stack的反向操作，将列索引变为行索引。默认是从右边索引开始变。

下面说一下参数：对于level就是转移行索引，默认是-1，也就上面说的从右往左转移。第二个参数fill_value也很容易猜到，前面stack的dropna是删除缺失值，这里的fill_value就是将出现的缺失值补充成NaN，默认为None。

参考学习：https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html#pandas.DataFrame.unstack

In [58]:
result = df_stacked.unstack().swaplevel(1,0,axis=1).sort_index(axis=1)
# 这里swaplevel是交换这两个列索引的位置，sort_index是将列索引分组。可以将上面的这两个函数去掉再运行，就能发现出差别了。
result.groupby('Class').head(2)

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
C_2   1201    NaN  188.0    NaN  68.0
      1202  176.0    NaN   94.0   NaN
C_3   1301    NaN  161.0    NaN  68.0
      1302  175.0    NaN   57.0   NaN
C_4   2401  192.0    NaN   62.0   NaN
      2402    NaN  166.0    NaN  82.0

In [56]:
result.equals(df_s)

True

## 三、哑变量与因子化
### 1、Dummy Variable（哑变量）
这里主要介绍get_dummies函数，其功能主要是进行one-hot编码：

In [59]:
df_d = df[['Class','Gender','Weight']]
df_d.head()

,Class,Gender,Weight
0,C_1,M,63
1,C_1,F,73
2,C_1,M,82
3,C_1,F,81
4,C_1,F,64


现在希望将上面的表格前两列转化为哑变量，并加入第三列Weight数值：

In [61]:
pd.get_dummies(df_d[['Class','Gender']]).join(df_d['Weight']).head()

,Class_C_1,Class_C_2,Class_C_3,Class_C_4,Gender_F,Gender_M,Weight
0,1,0,0,0,0,1,63
1,1,0,0,0,1,0,73
2,1,0,0,0,0,1,82
3,1,0,0,0,1,0,81
4,1,0,0,0,1,0,64


可选prefix参数添加前缀，prefix_sep添加分隔符,示例如下：

In [64]:
df_pp = df_d[['Class','Gender']]
pd.get_dummies(df_pp, prefix=['C','G'], prefix_sep='*').join(df_d['Weight']).head()

,C*C_1,C*C_2,C*C_3,C*C_4,G*F,G*M,Weight
0,1,0,0,0,0,1,63
1,1,0,0,0,1,0,73
2,1,0,0,0,0,1,82
3,1,0,0,0,1,0,81
4,1,0,0,0,1,0,64


这里prefix的默认是原前缀，prefix_sep默认为'_'。

参考学习：https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html?highlight=get_dummi

### 2. factorize方法
该方法主要用于自然数编码，并且缺失值会被记做-1，其中sort参数表示是否排序后赋值，默认为False。

In [66]:
codes, uniques = pd.factorize(['b', None, 'a', 'c', 'b'], sort=True)
display(codes)
display(uniques)

array([ 1, -1,  0,  2,  1], dtype=int64)

array(['a', 'b', 'c'], dtype=object)

codes是对元素进行编码，None为-1。 uniques得到列表的唯一元素s。

参考学习：https://pandas.pydata.org/docs/reference/api/pandas.factorize.html?highlight=factori#pandas.factorize

## 四、问题与练习
### 1、问题
#### 【问题一】 上面提到了许多变形函数，如melt/crosstab/pivot/pivot_table/stack/unstack函数，请总结它们各自的使用特点。

我们分拨来说：

1）首先我们讲 pivot、pivot_tabel，这两个变形函数都是对某列的元素变成列索引，功能很强大，可以同时计算平均值、总和等等数据，但是前者有一定的局限性。

2）其次说一下crosstab，这个函数可以计算频数，也可以计算百分比，功能也较为强大。

3）最后看这个melt、stack和unstack。这些函数主要就是用来变换行列索引，功能比较局限，其中stack的功能就是将行索引变成列索引，然后melt和unstack的功能类似，和stack的功能恰恰相反。

这里说的比较宽泛，还有很多参数会影响这些功能的使用，详细的就看上面的代码和链接吧。

#### 【问题二】 变形函数和多级索引是什么关系？哪些变形函数会使得索引维数变化？具体如何变化？

一般我们使用变形函数，会是变换行列索引，那么这里就会遇到这个多级索引的问题，到底换哪一个索引，怎么选择索引就值得我们来探讨。

从我们所学的来看，能使用多级索引的变形函数是pivot_tabel，这个函数功能很强大，行列和值都可以多级。那么面对这个多级索引，我们要变化维数，就要使用stack和unstack这些函数了。在这些函数中有专门的参数来代表我们要换的那一行列索引的位置level，从而实现选择索引。

#### 【问题三】 请举出一个除了上文提过的关于哑变量方法的例子。

下面我们改变df_d中的元素。

In [69]:
df_d = df[['School','Gender','Height']]
df_d.head()

,School,Gender,Height
0,S_1,M,173
1,S_1,F,192
2,S_1,M,186
3,S_1,F,167
4,S_1,F,159


In [70]:
pd.get_dummies(df_d[['School','Gender']]).join(df_d['Height']).head()

,School_S_1,School_S_2,Gender_F,Gender_M,Height
0,1,0,0,1,173
1,1,0,1,0,192
2,1,0,0,1,186
3,1,0,1,0,167
4,1,0,1,0,159


In [73]:
pd.get_dummies(df_d[['School','Gender']], prefix=['S','G'], prefix_sep='%').join(df_d['Height']).head()

,S%S_1,S%S_2,G%F,G%M,Height
0,1,0,0,1,173
1,1,0,1,0,192
2,1,0,0,1,186
3,1,0,1,0,167
4,1,0,1,0,159


#### 【问题四】 使用完stack后立即使用unstack一定能保证变化结果与原始表完全一致吗？

不一定。这两个变形函数都是有参数的，我们如果不考虑参数，遇到多级索引就很有可能不会一致。但是我们要是考虑参数，换的行正好对应换的列，然后通过参数找出，再换回来，再通过swaplevel和sort_index等函数进行修正，就可以做到一致。

#### 【问题五】 透视表中涉及了三个函数，请分别使用它们完成相同的目标（任务自定）并比较哪个速度最快。

In [78]:
%timeit df.pivot(index='ID',columns='Gender',values='Height')
%timeit pd.pivot_table(df,index='ID',columns='Gender',values='Height')
%timeit pd.crosstab(index=df['ID'],columns=df['Gender'], values=df['Height'], aggfunc='min')

1.86 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
9.21 ms ± 613 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.5 ms ± 882 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


最快的还是pivot函数。

#### 【问题六】 既然melt起到了unstack的功能，为什么再设计unstack函数？

虽然说melt和unstack很像，但是使用起来却十分的复杂，参数太多了，需要我们自己填写的东西很多。而这个unstack的参数就两个，level和fill_value，简单快捷，使用很方便。所以设计unstack函数应该是为了更方便的完成任务吧。

### 2、练习
#### 【练习一】 继续使用上一章的药物数据集：

In [67]:
pd.read_csv('joyful-pandas-master/data/Drugs.csv').head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


#### (a) 现在请你将数据表转化成如下形态，每行需要显示每种药物在每个地区的10年至17年的变化情况，且前三列需要排序：
![avatar](joyful-pandas-master/picture/drug_pic.png)

In [146]:
df = pd.read_csv('joyful-pandas-master/data/Drugs.csv',index_col=['State','COUNTY']).sort_index()
df.head()

YYYY  SubstanceName  DrugReports
State COUNTY                                  
KY    ADAIR   2010      Methadone            1
      ADAIR   2010    Hydrocodone            6
      ADAIR   2011      Oxycodone            4
      ADAIR   2011  Buprenorphine            3
      ADAIR   2011       Morphine            2

In [123]:
#result = pivoted.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math').dropna().set_index('ID').sort_index()

df_result = pd.pivot_table(df,index=['State','COUNTY','SubstanceName'],columns='YYYY',values='DrugReports',
                           fill_value='-').sort_index().reset_index().rename_axis(columns={'YYYY':''})
df_result.head()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3


#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [155]:
result_melted = result.melt(id_vars=result.columns[:3],value_vars=result.columns[-8:]
                ,var_name='YYYY',value_name='DrugReports').query('DrugReports != "-"')
result2 = result_melted.sort_values(by=['State','COUNTY','YYYY','SubstanceName']).reset_index().drop(columns='index')
#下面其实无关紧要，只是交换两个列再改一下类型（因为‘-’所以type变成object了）
cols = list(result2.columns)
a, b = cols.index('SubstanceName'), cols.index('YYYY')
cols[b], cols[a] = cols[a], cols[b]
result2 = result2[cols].astype({'DrugReports':'int','YYYY':'int'})
result2

,State,COUNTY,YYYY,SubstanceName,DrugReports
0,KY,ADAIR,2010,Hydrocodone,6
1,KY,ADAIR,2010,Methadone,1
2,KY,ADAIR,2011,Buprenorphine,3
3,KY,ADAIR,2011,Hydrocodone,9
4,KY,ADAIR,2011,Morphine,2
...,...,...,...,...,...
24057,WV,WYOMING,2013,Oxycodone,12
24058,WV,WYOMING,2014,Hydrocodone,1
24059,WV,WYOMING,2014,Oxycodone,5
24060,WV,WYOMING,2016,Hydrocodone,1


In [156]:
df_tidy = df.reset_index().sort_values(by=result2.columns[:4].tolist()).reset_index().drop(columns='index')
df_tidy

,State,COUNTY,YYYY,SubstanceName,DrugReports
0,KY,ADAIR,2010,Hydrocodone,6
1,KY,ADAIR,2010,Methadone,1
2,KY,ADAIR,2011,Buprenorphine,3
3,KY,ADAIR,2011,Hydrocodone,9
4,KY,ADAIR,2011,Morphine,2
...,...,...,...,...,...
24057,WV,WYOMING,2013,Oxycodone,12
24058,WV,WYOMING,2014,Hydrocodone,1
24059,WV,WYOMING,2014,Oxycodone,5
24060,WV,WYOMING,2016,Hydrocodone,1


In [157]:
df_tidy.equals(result2)

False

#### 【练习二】 现有一份关于某地区地震情况的数据集，请解决如下问题：

In [68]:
pd.read_csv('joyful-pandas-master/data/Earthquake.csv').head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


#### (a) 现在请你将数据表转化成如下形态，将方向列展开，并将距离、深度和烈度三个属性压缩：
![avatar](joyful-pandas-master/picture/earthquake_pic.png)

In [176]:
df = pd.read_csv('joyful-pandas-master/data/Earthquake.csv')
df = df.sort_values(by=df.columns.tolist()[:3]).sort_index(axis=1).reset_index().drop(columns='index')
df.head()

,方向,日期,时间,深度,烈度,经度,维度,距离
0,south_east,1912.08.09,12:29:00 AM,16.0,6.7,27.2,40.6,4.3
1,south_west,1912.08.10,12:23:00 AM,15.0,6.0,27.1,40.6,2.0
2,south_west,1912.08.10,12:30:00 AM,15.0,5.2,27.1,40.6,2.0
3,south_east,1912.08.11,12:19:04 AM,30.0,4.9,27.2,40.6,4.3
4,south_west,1912.08.11,12:20:00 AM,15.0,4.5,27.1,40.6,2.0


In [177]:
result = pd.pivot_table(df,index=['日期','时间','维度','经度'],columns='方向',values=['烈度','深度','距离'],
                        fill_value='-').stack(level=0).rename_axis(index={None:'地震参数'})
result.head(6)

方向                                    east north north_east north_west south  \
日期         时间          维度   经度   地震参数                                          
1912.08.09 12:29:00 AM 40.6 27.2 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   
                                 距离      -     -          -          -     -   
1912.08.10 12:23:00 AM 40.6 27.1 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   
                                 距离      -     -          -          -     -   

方向                                    south_east south_west west  
日期         时间          维度   经度   地震参数                             
1912.08.09 12:29:00 AM 40.6 27.2 深度           16          -    -  
                                 烈度          6.7          -    -  
                                 距离          4.3          -    -  
1912.08.10 12:23:00 AM 40.6 27.1 深度            -         15    -  
                                 烈度            -          6    -  
                                 距离            -          2    -

#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [178]:
df_result = result.unstack().stack(0)[(~(result.unstack().stack(0)=='-')).any(1)].reset_index()
df_result.columns.name=None
df_result = df_result.sort_index(axis=1).astype({'深度':'float64','烈度':'float64','距离':'float64'}).sort_index()
df_result.head()

,方向,日期,时间,深度,烈度,经度,维度,距离
0,south_east,1912.08.09,12:29:00 AM,16.0,6.7,27.2,40.6,4.3
1,south_west,1912.08.10,12:23:00 AM,15.0,6.0,27.1,40.6,2.0
2,south_west,1912.08.10,12:30:00 AM,15.0,5.2,27.1,40.6,2.0
3,south_east,1912.08.11,12:19:04 AM,30.0,4.9,27.2,40.6,4.3
4,south_west,1912.08.11,12:20:00 AM,15.0,4.5,27.1,40.6,2.0


In [179]:
df_result.astype({'深度':'float64','烈度':'float64','距离':'float64'},copy=False).dtypes

方向     object
日期     object
时间     object
深度    float64
烈度    float64
经度    float64
维度    float64
距离    float64
dtype: object

In [180]:
df.equals(df_result)

True